In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Загрузка датасета

In [35]:
df = pd.read_csv('vtb_preprocessing_dataset_target.csv', sep=';', encoding='cp1251')

df['month_date_open_pc'] = (df
                             .astype('str')
                             .date_open_pc
                             .apply(lambda x: x[:8] + '01')
                             .astype('datetime64')                    
                           )

df = df.loc[lambda x: x.inn!='AGLSEBFWDV'] #Возможно тестовый клиент -44 счета. Уберем

In [36]:
df.inn.value_counts()

QWZGVJAQVX      15
NUYSHDQNDH      12
EEOUKZVZHU      10
HAAKULNCVKVB    10
VSGGQVPMIM      10
                ..
LHCFGPHCPSFV     1
LMZVULWGLH       1
UNROJJODTKYR     1
HLLQEQKCSRWK     1
RZRKGVVHJX       1
Name: inn, Length: 65046, dtype: int64

In [37]:
pd.set_option('max_columns', 100)

print(df.shape)
df.head()

(67099, 34)


,inn,org_type,date_open_pc,target,Количество компаний с аналогичным директором в том же регионе,"Количество компаний, зарегистрированных на адресе регистрации Организации по данным сайта ФНС",Количество видов деятельности у Клиента,код основного оквэд,Субъект местонахождения,размер уставного капитал ЮЛ,Чистая прибыль (или убыток) компании,Отчетный период (год),Пассивы всего,дата начала полномочий руководителя,Количество соучредителей,Даты внесения соучредителей,Дата регистрации,Количество филиалов,Численность компании,"Выручка компании (млн, руб)",Участие в госконтрактах (количество),Участие в госконтрактах (год),Сумма налога,ОПФ Организации,"7 - массовый контактный телефон: в заявке указан телефон, на который зарегистрировано более Х компаний (кроме компаний-агентов)",8 - Количество компаний с аналогичным директором,8 - Количество компаний с аналогичным директором в том же регионе,25 - Государственные заказы и контракты Приняла участие (количество),25 - Коммерческие заказы и контракты Количество заключенных контрактов,Индекс финансового риска описание,Индекс должной осмотрительности описание,История смены сооучредителей,is_credit,month_date_open_pc
0,QZSCOUYDPK,ЮЛ,2019-03-13 00:00:00,0,2.0,1.0,10.0,NaN,NaN,10000.0,NaN,NaN,NaN,24.01.2018,1.0,24.01.2018,24.01.2018,0.0,NaN,NaN,NaN,NaN,NaN,65-Общества с ограниченной ответственностью,1.0,2.0,2.0,NaN,NaN,Средний риск,Средний риск,ЮЛ: 24.01.2018 0:00:00,0.0,2019-03-01
1,XGJYAHPQJUMT,ИП,2019-02-18 00:00:00,0,0.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.10.2018,1.0,NaN,NaN,NaN,NaN,NaN,50102-Индивидуальные предприниматели,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,2019-02-01
2,JGRZVAAXTH,ЮЛ,2019-03-29 00:00:00,0,3.0,3.0,18.0,NaN,NaN,19600.0,7445830.0,2017.0,34270600.0,13.05.2011;27.09.2007,3.0,13.01.2014;13.05.2011;01.02.2010,27.09.2007,0.0,16 .. 50,30.5,NaN,NaN,NaN,65-Общества с ограниченной ответственностью,29.0,3.0,3.0,2.0,NaN,Низкий риск,Средний риск,ЮЛ: 13.01.2014 0:00:00; ЮЛ: 13.05.2011 0:00:00...,0.0,2019-03-01
3,CQNMOOTKHQAN,ИП,2018-11-15 00:00:00,0,0.0,NaN,5.0,NaN,AJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09.11.2018,1.0,NaN,NaN,NaN,NaN,NaN,50102-Индивидуальные предприниматели,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,2018-11-01
4,RDDXQARZUE,ЮЛ,2019-03-29 00:00:00,0,2.0,11.0,3.0,NaN,NaN,260000.0,3784950.0,2017.0,1063230.0,26.11.2013;08.11.2012;09.12.2011;22.10.2008;15...,2.0,05.09.2013;09.12.2011;22.10.2008,15.11.2007,0.0,16 .. 50,44.4,0.0,2017.0,NaN,65-Общества с ограниченной ответственностью,1.0,2.0,2.0,0.0,0.0,Низкий риск,Низкий риск,ЮЛ: 05.09.2013 0:00:00; ЮЛ: 09.12.2011 0:00:00...,0.0,2019-03-01


In [38]:
(
  df
    .groupby('month_date_open_pc')['target']\
     .agg(['sum', 'mean', 'count'])\
     .reset_index()\
     .assign(percent_target=lambda x: round(100*(x['sum'] / df.target.sum()), 1))
     .rename(columns={'month_date_open_pc': 'Месяц открытия счета',
                      'sum': 'Количество таргетов',
                      'mean': 'Доля таргетов',
                      'count': 'Всего наблюдений',
                      'percent_target': 'Доля таргетов от всех'})
)
#.loc[lambda x: x['Месяц открытия счета']>=pd.to_datetime('2019-09-01')]

,Месяц открытия счета,Количество таргетов,Доля таргетов,Всего наблюдений,Доля таргетов от всех
0,2018-06-01,9,0.006374,1412,0.1
1,2018-07-01,20,0.004769,4194,0.3
2,2018-08-01,65,0.017483,3718,0.9
3,2018-09-01,106,0.030654,3458,1.5
4,2018-10-01,275,0.054402,5055,3.9
5,2018-11-01,645,0.100750,6402,9.2
6,2018-12-01,930,0.142660,6519,13.3
7,2019-01-01,645,0.139490,4624,9.2
8,2019-02-01,1215,0.149336,8136,17.4
9,2019-03-01,1280,0.131363,9744,18.3


### Доля блокировок за последние 4 месяца

In [15]:
(33+23+18+14) / (325+312+305+268)

0.07272727272727272

### Доля блокировок за последние 6 месяцев

In [16]:
(48+39+33+23+18+14) / (327+298+325+312+305+268)

0.09536784741144415

# Деление на train и test

### Разделение на выборки
### valid_2 - данные с 2019-05-01. По которым мало spark данных - новые заявки

In [39]:
valid_2 = df[df['month_date_open_pc']>=pd.to_datetime('2019-05-01')]

### valid_1 - отложенная выборка от train_test без клиентов из valid_2

Уберем клиентов, которые были в valid_1

In [40]:
sample = df.loc[lambda x: x.month_date_open_pc < pd.to_datetime('2019-05-01')]\
           .loc[lambda x: ~x.inn.isin(valid_2.inn)]

In [41]:
X_, X_valid_1, y_, y_valid_1 = train_test_split(sample.drop('target', axis=1), 
                                                sample['target'],
                                                random_state=12,
                                                stratify=sample['target'], 
                                                test_size=0.1)

valid_1 = pd.concat([X_valid_1, y_valid_1], axis=1)

### train_test -выборка с 2018-05 по 2019-04 без клиентов из valid_1 и valid_2

In [42]:
train_test = df.loc[lambda x: x.month_date_open_pc < pd.to_datetime('2019-09-01')]\
                .loc[lambda x: ~x.inn.isin(valid_2.inn)]\
                .loc[lambda x: ~x.inn.isin(valid_1.inn)]

## Статистика по собранным выборкам

In [43]:
def get_statistics(df, sample_name):
    
    print(sample_name)
    print('Размерность:', df.shape[0])
    print('Таргетов', df.target.sum())
    print('Дисбаланс:', df.target.sum() / df.shape[0])
    print('-------------')
    

In [44]:
get_statistics(train_test, 'train_test')
get_statistics(valid_1, 'valid_1')
get_statistics(valid_2, 'valid_2')

train_test
Размерность: 56645
Таргетов 5912
Дисбаланс: 0.10436931768028952
-------------
valid_1
Размерность: 6324
Таргетов 658
Дисбаланс: 0.10404807084123972
-------------
valid_2
Размерность: 3114
Таргетов 405
Дисбаланс: 0.13005780346820808
-------------


## Сохранение

In [45]:
valid_2.columns = [col.replace(' ','_') for col in valid_2.columns]
valid_1.columns = [col.replace(' ','_') for col in valid_1.columns]
train_test.columns = [col.replace(' ','_') for col in train_test.columns]

In [46]:
train_test.to_csv('train_test.csv', index=False, sep=';', encoding='cp1251')
valid_1.to_csv('valid_1.csv', index=False, sep=';', encoding='cp1251')
valid_2.to_csv('valid_2.csv', index=False, sep=';', encoding='cp1251')